# Intermediate Tutorial: Composition and Navigation

This tutorial covers composing prompts from smaller pieces, navigating nested structures, and exporting provenance data.

In [1]:
from t_prompts import prompt

## Nested Prompts

You can build larger prompts by composing smaller `StructuredPrompt` objects together.

In [2]:
# Build prompts from smaller pieces
system_msg = "You are a helpful assistant."
user_query = "What is Python?"

p_system = prompt(t"{system_msg:system}")
p_user = prompt(t"User: {user_query:query}")

# Compose into larger prompt
p_full = prompt(t"{p_system:sys} {p_user:usr}")

# Renders correctly
print(str(p_full))

You are a helpful assistant. User: What is Python?


## Navigating Nested Structures

Access nested interpolations using chained subscript operations.

In [3]:
# Navigate the tree
print(f"System message: {p_full['sys']['system'].value}")
print(f"User query: {p_full['usr']['query'].value}")

# Get the nested prompt itself
nested = p_full['sys'].value
print(f"\nNested prompt type: {type(nested)}")
print(f"Nested prompt text: {str(nested)}")

System message: You are a helpful assistant.
User query: What is Python?

Nested prompt type: <class 't_prompts.core.StructuredPrompt'>
Nested prompt text: You are a helpful assistant.


## Conversions: !s, !r, !a

t-strings support conversion flags from Python's string formatting.

In [4]:
text = "Hello\nWorld"

# !s: str() conversion (default)
p_s = prompt(t"{text!s:s}")
print(f"!s: {str(p_s)}")

# !r: repr() conversion
p_r = prompt(t"{text!r:r}")
print(f"!r: {str(p_r)}")

# !a: ascii() conversion
emoji = "Hello 👋"
p_a = prompt(t"{emoji!a:a}")
print(f"!a: {str(p_a)}")

# Conversion is preserved in metadata
print(f"\nConversion for 'r': {p_r['r'].conversion}")

!s: Hello
World
!r: 'Hello\nWorld'
!a: 'Hello \U0001f44b'

Conversion for 'r': r


## Mapping Protocol

`StructuredPrompt` implements the mapping protocol: `keys()`, `values()`, `items()`, `get()`, etc.

In [5]:
name = "Alice"
age = "30"
city = "NYC"

p = prompt(t"Name: {name:n}, Age: {age:a}, City: {city:c}")

# Keys
print(f"Keys: {list(p.keys())}")

# Values (returns StructuredInterpolation nodes)
print("\nValues:")
for node in p.values():
    print(f"  {node.key}: {node.value}")

# Items
print("\nItems:")
for key, node in p.items():
    print(f"  {key} -> {node.expression} = {node.value}")

# get() with default
print(f"\nget('n'): {p.get('n').value}")
print(f"get('missing'): {p.get('missing')}")

Keys: ['n', 'a', 'c']

Values:
  n: Alice
  a: 30
  c: NYC

Items:
  n -> name = Alice
  a -> age = 30
  c -> city = NYC

get('n'): Alice
get('missing'): None


## Exporting to Values

Use `to_values()` to export just the key-value pairs (useful for logging or serialization).

In [6]:
import json

context = "User is Alice"
instructions = "Be concise"

p = prompt(t"Context: {context:ctx}. {instructions:inst}")

# Export to dictionary
values = p.to_values()
print(json.dumps(values, indent=2))

{
  "ctx": "User is Alice",
  "inst": "Be concise"
}


## Exporting Full Provenance

Use `to_provenance()` to export complete metadata including expressions, conversions, and format specs.

In [7]:
# Export full provenance
provenance = p.to_provenance()
print(json.dumps(provenance, indent=2))

{
  "strings": [
    "Context: ",
    ". ",
    ""
  ],
  "nodes": [
    {
      "key": "ctx",
      "expression": "context",
      "conversion": null,
      "format_spec": "ctx",
      "render_hints": "",
      "index": 1,
      "value": "User is Alice"
    },
    {
      "key": "inst",
      "expression": "instructions",
      "conversion": null,
      "format_spec": "inst",
      "render_hints": "",
      "index": 3,
      "value": "Be concise"
    }
  ]
}


## Nested Provenance

Provenance export works recursively for nested prompts.

In [8]:
# Create nested structure
inner = prompt(t"{context:ctx}")
outer = prompt(t"Outer: {inner:in}. {instructions:inst}")

# Export nested provenance
nested_prov = outer.to_provenance()
print(json.dumps(nested_prov, indent=2))

{
  "strings": [
    "Outer: ",
    ". ",
    ""
  ],
  "nodes": [
    {
      "key": "in",
      "expression": "inner",
      "conversion": null,
      "format_spec": "in",
      "render_hints": "",
      "index": 1,
      "value": {
        "strings": [
          "",
          ""
        ],
        "nodes": [
          {
            "key": "ctx",
            "expression": "context",
            "conversion": null,
            "format_spec": "ctx",
            "render_hints": "",
            "index": 1,
            "value": "User is Alice"
          }
        ]
      }
    },
    {
      "key": "inst",
      "expression": "instructions",
      "conversion": null,
      "format_spec": "inst",
      "render_hints": "",
      "index": 3,
      "value": "Be concise"
    }
  ]
}


## Introspection

You can inspect the structure of a prompt to understand its composition.

In [9]:
def inspect_prompt(p, indent=0):
    """Recursively inspect a structured prompt."""
    prefix = "  " * indent
    for key, node in p.items():
        if isinstance(node.value, str):
            print(f"{prefix}{key}: {node.expression} = \"{node.value}\"")
        else:  # Nested StructuredPrompt
            print(f"{prefix}{key}: {node.expression} (nested)")
            inspect_prompt(node.value, indent + 1)

inspect_prompt(outer)

in: inner (nested)
  ctx: context = "User is Alice"
inst: instructions = "Be concise"


## Next Steps

Explore the topic notebooks for deep dives:

- **Few-Shot Prompts**: Building lists of examples with dynamic keys
- **Source Mapping**: Bidirectional text ↔ structure mapping
- **Format Spec Mini-Language**: Custom keys and render hints
- **Advanced Composition**: Complex nested structures and patterns